In [ ]:
library(readxl)
library(vegan)
library(ggplot2)
library(SRS)
library(tidyverse)
library(divermeta)
library(kableExtra)
library(FSA)
library(latex2exp)
library(cowplot)
library(patchwork)
library(gridExtra)
library(pheatmap)
library(multcompView)

source("utils/general_functions.R")

In [ ]:


# Define the file path and sheet name
file_path <- "LUCAS/LUCAS_ARGs_for_Multiplicity.xlsx"
sheet_name <- "ARGs_hierarchy"

depth_file <- "LUCAS/seq_depth.xlsx" 

# Read data from line 4 to 64
df_clust <- read_excel(path = file_path, sheet = sheet_name)

df_meta <- read_excel(path = file_path, sheet = "Sample_metadata")

df_depth <- read_excel(path = depth_file) 


levels <- c("Gene", "Family", "Subclass", "Class")
sheets <- c("Gene" = "ARGs_abund_Gene", 'Family' = "ARGs_abund_Family", "Subclass" = "ARG_abund_Subclass", "Class" = "ARG_abund_Class")


l1 <- "Gene"
l2 <- "Subclass"

df_ab <- read_excel(path = file_path, sheet = sheets[l1])


In [ ]:
# Computes the missing reads for each sample
df_temp <- df_ab %>% select(SampleID)
tot <- rowSums(df_ab %>% select(-SampleID))

df_temp["actual"] <- tot 

df_depth <- inner_join(df_depth, df_temp, by = "SampleID")

df_depth["missing"] <- df_depth$Total_functional_reads - df_depth$actual


# Adds the mssing
df_ab <- inner_join(df_ab, df_depth %>% select(SampleID, missing), by = "SampleID")


min_depth <- df_depth %>% filter(Total_functional_reads > 100)
min_depth <- min(min_depth$Total_functional_reads)
min_depth

In [ ]:

df_filtered_ab <- df_ab %>% 
  inner_join(df_depth %>% filter(Total_functional_reads >= min_depth) %>% select(SampleID), by = "SampleID")

df_filtered_ab

## SRS

In [ ]:
set.seed(1)

otus <- colnames(df_filtered_ab  %>% select(-SampleID))
samples_ids  <- df_filtered_ab$SampleID

# sets up the structure
df_normalized  <-  as_tibble(t(df_filtered_ab  %>% select(-SampleID)))
colnames(df_normalized)  <- samples_ids
# Normalizes
normalized <- SRS(df_normalized, min_depth)
sample_ids <- colnames(normalized)

# Returns to the structure
df_normalized  <- as_tibble(t(normalized))
colnames(df_normalized)  <- otus
df_normalized <- cbind(data.frame(SampleID = sample_ids), df_normalized)

# Removes the Missing
df_normalized  <-  df_normalized  %>%  select(-missing)

nrow(df_normalized)

## Multiplicity and Diversity

In [ ]:
mul <- c()
div <- c()



# Iterates over each row
for(sample in df_normalized$SampleID)
{
    df_ab_sample <- df_normalized[df_normalized$SampleID == sample,]
    genes <- colnames(df_ab_sample)
    df_ab_sample <- data.frame(t(df_ab_sample))
    df_ab_sample["Gene"] <-genes
    colnames(df_ab_sample) <- c("abundance", "Gene")
    df_ab_sample <- df_ab_sample[-1,]

    df_ab_sample <- inner_join(df_ab_sample, df_clust[,c("Gene", l2)], by ="Gene")
    df_ab_sample$abundance = as.integer(df_ab_sample$abundance)
    df_ab_sample <- df_ab_sample[df_ab_sample$abundance > 0,]

    ab = df_ab_sample$abundance
    clust = df_ab_sample[[l2]]
    
    # Post Clust
    ab_clust_sample <- tapply(ab, clust, sum)
    p_clust = ab_clust_sample /sum(ab_clust_sample)
    div_clust <- exp(-sum(p_clust*log(p_clust)))
    
    div <- c(div, div_clust) 
    mul <- c(mul, multiplicity.inventory(ab, clust))

}


## Kruskal-Wallis Test


In [ ]:


set.seed(1)

# Short Label Order
short_label_order = c("WC", "WD", "Gr", "Cr")
label_order = c("woodland_coniferous", "woodland_deciduous", "grassland", "cropland")

# Prepare dataframe
df_analysis <- data.frame(SampleID = df_filtered_ab$SampleID, div = div, mul = mul)
df_analysis <- inner_join(df_analysis, df_meta, by = "SampleID")

multiplicity_col <- "mul"
diversity_col <- "div"
group_col <- "LandCover"

multiplicity_name <- "Inventory Multiplicity $^1M$"
diversity_name <- "Diversity $^1D$"
group_name <- "Habitat Type"

kw_caption <- "Kruskal-Wallis test results for diversity ($^1D$) and inventory multiplicity ($^1M$) comparing the different habitat type"
dun_caption <- "Dunn's Test Results for diversity ($^1D$) and inventory multiplicity ($^1M$) comparing the different habitat type"

      
kw_label <- "tab:kw_results_antibiotics"
dunn_label <- "tab:dunn_results_antibiotics"



tests_resp <- kw_and_dunn_test(
        df_test = df_analysis,
        order = label_order,
        labels = short_label_order, 
        multiplicity_col = multiplicity_col,
        diversity_col = diversity_col,
        group_col = group_col,
        diversity_name = diversity_name,
        group_name = group_name,
        multiplicity_name = multiplicity_name,
        kw_caption = kw_caption,
        dun_caption = dun_caption,
        kw_label = kw_label,
        dunn_label = dunn_label
      )


cat(tests_resp$kw_table)
cat("\n\n")
cat(tests_resp$dunn_table)




## Plots

In [ ]:

# Set plot size
options(repr.plot.width = 14, repr.plot.height = 14)


# Parameterrs
df <- data.frame(SampleID = df_filtered_ab$SampleID, div = div, mul = mul)
df <- df %>% inner_join(df_meta, by = "SampleID")




label_map <- c(
  "woodland_coniferous" = "Woodland Coniferous",
  "woodland_deciduous" = "Woodland Deciduous",
  "grassland" = "Grassland",
  "cropland" = "Cropland"
)

short_label_map <- c(
  "woodland_coniferous" = "WC",
  "woodland_deciduous" = "WD",
  "grassland" = "Gr",
  "cropland" = "Cr"
)

custom_colors <- c( 
                    '#7D917E',                 
                    '#5A7273',
                    '#DAC5AA',
                    '#BE5E58'
                    )


title <- TeX(paste0("Inventory Multiplicity $^1M$ vs ",l2, " Diversity $^1D$"))


plot_location <- "LUCAS/plots/srs_merged_plot.png"

cld_div <- compute_cld(dunn_result = tests_resp$dunn_div, short_label_order = short_label_order, label_order = label_order)
cld_mul <- compute_cld(dunn_result = tests_resp$dunn_mul, short_label_order = short_label_order, label_order = label_order)


final_plot <- plot_scatter_and_box_plot(  
  df= df,
  multiplicity_col= multiplicity_col,
  diversity_col= diversity_col,
  group_col= group_col,
  label_map= label_map,  
  short_label_map = short_label_map,
  custom_colors= custom_colors,
  title= title,
  multiplicity_name= TeX(multiplicity_name),
  diversity_name= TeX(diversity_name),
  group_name= group_name,
  plot_location= plot_location,
  cld_div= cld_div,
  cld_mul= cld_mul
)

final_plot

## Exploration by ARG (Class and Subclass)


In [ ]:
# Iterates over each row

target_level  <- "Subclass"


results  <- c()
for(sample in df_normalized$SampleID)
{

    print(sample)
    flush.console()

    df_ab_sample <- df_normalized[df_normalized$SampleID == sample,]
    genes <- colnames(df_ab_sample)
    df_ab_sample <- data.frame(t(df_ab_sample))
    df_ab_sample["Gene"] <-genes
    colnames(df_ab_sample) <- c("abundance", "Gene")
    df_ab_sample <- df_ab_sample[-1,]

    df_ab_sample <- inner_join(df_ab_sample, df_clust[,c("Gene", target_level)], by ="Gene")
    df_ab_sample$abundance  <-  as.integer(df_ab_sample$abundance)
    df_ab_sample <- df_ab_sample[df_ab_sample$abundance > 0,]

    # Gets the LandCover 
    lc  <- df_meta %>% filter(SampleID == sample) %>% pull(LandCover) 



    for(group in unique(df_ab_sample[[target_level]]))
    {

        df_temp  <- df_ab_sample %>% filter(.data[[target_level]] == group)
        ab  <-  df_temp$abundance
        clust <- df_temp  %>% pull(!!target_level)
        mul  <-  multiplicity.inventory(ab, clust)
        results  <-  rbind(results, c(group, lc, sample, mul, sum(ab)))
        


    }

    
}


df  <- as_tibble(results)
colnames(df) <- c("Group", "LandCover", "SampleID", "Multiplicity", "Abundance") 
df <- df %>% mutate(Multiplicity = as.numeric(Multiplicity), Abundance = as.numeric(Abundance))



In [ ]:
options(repr.plot.width = 15, repr.plot.height = 5)

min_abundance <- 200000




# Define custom colors
custom_colors <- c("#BE5E58", "#DAC5AA", "#5A7273", "#7D917E")
custom_colors <- setNames(custom_colors, c("cropland", "grassland", "woodland_coniferous", "woodland_deciduous"))


df_explore <- df %>% group_by(SampleID,LandCover) %>% 
summarise(TotalMultiplicity = weighted.mean(Multiplicity,Abundance)) %>% 
ungroup()


ggplot(df_explore, aes(x = LandCover, y = TotalMultiplicity, color = LandCover)) +
  geom_boxplot() +
  labs(
    # title = TeX("Inventory Multiplicity"),
    title = "All ARG",
    x = "Land Cover",
    y = TeX("Inventory Multiplicity $^1M$")
  )   +
  scale_x_discrete(labels = label_map) +
  scale_color_manual(values = custom_colors) +  # Add labels
      theme(
    axis.title = element_text(size = 18),   # Increase axis title font size
    axis.text = element_text(size = 16),    # Increase axis label font size
    plot.title = element_text(size = 20, hjust = 0.5),
    axis.text.y = element_blank(), # Remove x-axis text
    axis.ticks.y = element_blank(), 
    axis.title.y = element_blank() ,
      legend.text = element_text(size = 16),  # Increase legend text size
  legend.title = element_text(size = 18, face = "bold"),  # Increase legend title size
  legend.key.size = unit(1.5, "cm")  # Increase legend key size
  ) +
  coord_flip()   # Hides the legend

df_totals  <- df %>% 
              group_by(Group) %>% 
              summarise(totalAbundance = sum(Abundance)) %>%               
              arrange(-totalAbundance) %>% ungroup()

tot  <- sum(df_totals$totalAbundance)
df_totals <- df_totals %>% mutate(percentage = 100*totalAbundance/tot)              

selected_groups <- df_totals %>% filter(totalAbundance >= min_abundance) %>% pull(Group)




df_temp  <- df_totals %>% select(Group,totalAbundance,percentage)

head(df_temp, 7)

df_temp  <- df_temp %>% filter(totalAbundance >= min_abundance) %>% select(Group,totalAbundance,percentage)




df_explore <- df_temp %>% inner_join(df, by = "Group") %>% group_by(SampleID,LandCover) %>% 
summarise(TotalMultiplicity = weighted.mean(Multiplicity,Abundance)) %>% 
ungroup()


ggplot(df_explore, aes(x = LandCover, y = TotalMultiplicity, color = LandCover)) +
  geom_boxplot() +
  labs(
    # title = TeX("Inventory Multiplicity"),
    title = paste("Top",nrow(df_temp),"ARG"),
    x = "Land Cover",
    y = TeX("Inventory Multiplicity $^1M$")
  )   +
  scale_x_discrete(labels = label_map) +
  scale_color_manual(values = custom_colors) +  # Add labels
      theme(
    axis.title = element_text(size = 18),   # Increase axis title font size
    axis.text = element_text(size = 16),    # Increase axis label font size
    plot.title = element_text(size = 20, hjust = 0.5),
    axis.text.y = element_blank(), # Remove x-axis text
    axis.ticks.y = element_blank(), 
    axis.title.y = element_blank(),
      legend.text = element_text(size = 16),  # Increase legend text size
  legend.title = element_text(size = 18, face = "bold"),  # Increase legend title size
  legend.key.size = unit(1.5, "cm")  # Increase legend key size
  ) +
  coord_flip()   # Hides the legend


In [ ]:

target_level  <- "Subclass"


results  <- c()
for(sample in df_normalized$SampleID)
{

    print(sample)
    flush.console()

    df_ab_sample <- df_normalized[df_normalized$SampleID == sample,]
    genes <- colnames(df_ab_sample)
    df_ab_sample <- data.frame(t(df_ab_sample))
    df_ab_sample["Gene"] <-genes
    colnames(df_ab_sample) <- c("abundance", "Gene")
    df_ab_sample <- df_ab_sample[-1,]

    df_ab_sample <- inner_join(df_ab_sample, df_clust[,c("Gene", target_level)], by ="Gene")
    df_ab_sample$abundance  <-  as.integer(df_ab_sample$abundance)
    df_ab_sample <- df_ab_sample[df_ab_sample$abundance > 0,]

    # Gets the LandCover 
    lc  <- df_meta %>% filter(SampleID == sample) %>% pull(LandCover) 

    for(i in 1:length(selected_groups))
    {
        for(j in i:length(selected_groups))
        {
            group_1 <- selected_groups[i]
            group_2 <- selected_groups[j]

            group_label <- paste(group_1,"-",group_2)
            if(group_1 == group_2)
                group_label  <- group_1
            
            df_temp  <- df_ab_sample %>% filter(.data[[target_level]] == group_1 | .data[[target_level]] == group_2)
                                
            ab  <-  df_temp$abundance
            clust <- df_temp  %>% pull(!!target_level)
            mul  <-  multiplicity.inventory(ab, clust)
            results  <-  rbind(results, c(group_label, lc, sample, mul, sum(ab)))


        }
        
    }
    
}


df_pairs  <- as_tibble(results)
colnames(df_pairs) <- c("Group", "LandCover", "SampleID", "Multiplicity", "Abundance") 
df_pairs <- df_pairs %>% mutate(Multiplicity = as.numeric(Multiplicity), Abundance = as.numeric(Abundance))



In [ ]:
options(repr.plot.width = 13, repr.plot.height = 20)


df_plot <- df_pairs 

df_temp  <- df_pairs %>% select(Group, Abundance) %>% group_by(Group) %>% summarise(totalAbundance = sum(Abundance))  %>% arrange(-totalAbundance)

# Create a lookup table for Group labels with TotalAbundance
df_group_labels <- df_temp %>%
  distinct(Group, totalAbundance) %>%
  mutate(GroupLabel = Group, totalAbundance = formatC(totalAbundance, format = "f", big.mark = ",", digits = 0)) %>% 
  mutate(GroupLabel = str_replace_all(GroupLabel, "_", " "))  %>% 
  mutate(GroupLabel = str_replace_all(GroupLabel, ";", " "))  %>% 
  mutate(GroupLabel = str_replace_all(GroupLabel, ",", " "))  %>% 
  mutate(GroupLabel = str_wrap(GroupLabel, width = 10))  %>% 
  mutate(GroupLabel = paste0(GroupLabel, "\n(", totalAbundance, ")")) 



# Sorts
df_plot <- df_plot %>%
  mutate(Group = factor(Group, levels = df_temp$Group))


# Custom label mapping function
group_labeller <- function(variable, value) {
  setNames(df_group_labels$GroupLabel, df_group_labels$Group)[value]
}


# Define label mapping
label_map <- c(
  "woodland_deciduous" = "Woodland Deciduous",
  "woodland_coniferous" = "Woodland Coniferous",
  "grassland" = "Grassland",
  "cropland" = "Cropland"
)

# Define custom colors
custom_colors <- c("#BE5E58", "#DAC5AA", "#5A7273", "#7D917E")
custom_colors <- setNames(custom_colors, c("cropland", "grassland", "woodland_coniferous", "woodland_deciduous"))


# Calculate horizontal line positions dynamically
hline_positions <- df_plot %>%
  group_by(Group) %>%
  summarise(y_max = max(Multiplicity), .groups = "drop") %>%
  mutate(y_sep = lead(y_max))  # Get separation lines


ggplot(df_plot, aes(x = "", y = Multiplicity, fill = LandCover)) +  # Empty x-axis for boxplots
  geom_boxplot() +
     #geom_jitter(width = 0.1, alpha = 0.25, color = "#3A3A3A") +
  geom_hline(data = hline_positions, aes(yintercept = y_max), linetype = "dashed", color = "gray50", size = 0.7, na.rm = TRUE) +
  facet_grid(Group ~ LandCover, labeller = labeller(LandCover = label_map, Group = group_labeller), scales = "free_y") +  # Create a grid layout
    scale_fill_manual(values = custom_colors) +  # Apply custom colors

  labs(x = "Land Cover", y = "Multiplicity", title = paste("Multiplicity by",target_level ,"and Land Cover (", target_level,"comparisson )")) +
    theme_minimal(base_size = 16) +  # Increase text size
  theme(
    strip.text = element_text(size = 15),  # Larger facet labels
    axis.text = element_text(size = 12),   # Larger axis labels
    strip.text.y = element_text(size = 15, face = "bold", angle = 0, hjust = 0),  # Rotate labels horizontally
    plot.title = element_text(size = 18, face = "bold"),
    #legend.position = "none",
    legend.text = element_text(size = 16),  # Increase legend text size
  legend.title = element_text(size = 18, face = "bold"),  # Increase legend title size
  legend.key.size = unit(1.5, "cm") 
    )  # Bigger title

ggsave(paste0("LUCAS/plots/",target_level,"_",target_level,".jpg", sep=""), width = 18, height = 50, dpi = 400, limitsize = FALSE)


In [ ]:
options(repr.plot.width = 13, repr.plot.height = 20)


df_plot <- df_pairs 

df_temp  <- df_pairs %>% select(Group, Abundance) %>% group_by(Group) %>% summarise(totalAbundance = sum(Abundance))  %>% arrange(-totalAbundance)

# Create a lookup table for Group labels with TotalAbundance
df_group_labels <- df_temp %>%
  distinct(Group, totalAbundance) %>%
  mutate(GroupLabel = Group, totalAbundance = formatC(totalAbundance, format = "f", big.mark = ",", digits = 0)) %>% 
  mutate(GroupLabel = str_replace_all(GroupLabel, "_", " "))  %>% 
  mutate(GroupLabel = str_replace_all(GroupLabel, ";", " "))  %>% 
  mutate(GroupLabel = str_replace_all(GroupLabel, ",", " "))  %>% 
  mutate(GroupLabel = str_wrap(GroupLabel, width = 10))  %>% 
  mutate(GroupLabel = paste0(GroupLabel, "\n(", totalAbundance, ")")) 



# Sorts
df_plot <- df_plot %>%
  mutate(Group = factor(Group, levels = df_temp$Group))


# Custom label mapping function
group_labeller <- function(variable, value) {
  setNames(df_group_labels$GroupLabel, df_group_labels$Group)[value]
}


# Define label mapping
label_map <- c(
  "woodland_deciduous" = "Woodland Deciduous",
  "woodland_coniferous" = "Woodland Coniferous",
  "grassland" = "Grassland",
  "cropland" = "Cropland"
)

# Define custom colors
custom_colors <- c("#BE5E58", "#DAC5AA", "#5A7273", "#7D917E")
custom_colors <- setNames(custom_colors, c("cropland", "grassland", "woodland_coniferous", "woodland_deciduous"))


# Calculate horizontal line positions dynamically
hline_positions <- df_plot %>%
  group_by(Group) %>%
  summarise(y_max = max(Abundance), .groups = "drop") %>%
  mutate(y_sep = lead(y_max))  # Get separation lines


ggplot(df_plot, aes(x = "", y = Abundance, fill = LandCover)) +  # Empty x-axis for boxplots
  geom_boxplot() +
     #geom_jitter(width = 0.1, alpha = 0.25, color = "#3A3A3A") +
  geom_hline(data = hline_positions, aes(yintercept = y_max), linetype = "dashed", color = "gray50", size = 0.7, na.rm = TRUE) +
  facet_grid(Group ~ LandCover, labeller = labeller(LandCover = label_map, Group = group_labeller), scales = "free_y") +  # Create a grid layout
    scale_fill_manual(values = custom_colors) +  # Apply custom colors

  labs(x = "Land Cover", y = "", title = paste("Abundance by",target_level ,"and Land Cover (", target_level,"comparisson )")) +
    theme_minimal(base_size = 16) +  # Increase text size
  theme(
    strip.text = element_text(size = 15),  # Larger facet labels
    axis.text = element_text(size = 12),   # Larger axis labels
    strip.text.y = element_text(size = 15, face = "bold", angle = 0, hjust = 0),  # Rotate labels horizontally
    plot.title = element_text(size = 18, face = "bold"),
        #legend.position = "none",
          legend.text = element_text(size = 16),  # Increase legend text size
  legend.title = element_text(size = 18, face = "bold"),  # Increase legend title size
  legend.key.size = unit(1.5, "cm") )  # Bigger title

ggsave(paste0("LUCAS/plots/",target_level,"_",target_level,".jpg", sep=""), width = 18, height = 50, dpi = 400, limitsize = FALSE)


In [ ]:

df_temp  <- df_totals %>% filter(totalAbundance >= min_abundance) %>% select(Group,totalAbundance)

df_filtered <- df_temp %>% inner_join(df_plot, by = "Group")

options(repr.plot.width = 16, repr.plot.height = 16)
ps <- list()

for(lc in c("cropland", "grassland", "woodland_coniferous", "woodland_deciduous"))
{

  df_crosstab <- df_filtered %>%
  filter(LandCover == lc) %>% 
  select(SampleID, Group, Multiplicity) %>% 
    pivot_wider(names_from = Group, values_from = Multiplicity,values_fill = 0) %>%
    column_to_rownames("SampleID")


  cor_matrix <- cor(df_crosstab, use = "pairwise.complete.obs")  # Handle missing values


  # Convert correlation matrix into long format for ggplot
  cor_long <- as.data.frame(as.table(cor_matrix))

  # Plot the correlation heatmap
  p <-  ggplot(cor_long, aes(Var1, Var2, fill = Freq)) +
    geom_tile(color = "white") +  # White grid lines for clarity
      geom_text(aes(label = sprintf("%.2f", Freq)), color = "black", size = 5) +  # Add text label
    scale_fill_gradient2(low = "blue", mid = "white", high = "red", midpoint = 0) +  # Diverging colors
      scale_y_discrete(limits = rev(levels(cor_long$Var2))) +  # Reverse Y-axis order
    theme_minimal() +
    labs(title = label_map[lc],        
        x = "Group", y = "Group", fill = "Correlation") +
    theme(axis.text.x = element_text(angle = 45, hjust = 1),
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5),
    axis.text = element_text(size = 16))

  ps[[length(ps)+1]] = p

  print(paste(lc,":", mean(abs(cor_matrix[upper.tri(cor_matrix)]))))

  

}


do.call(grid.arrange, c(ps, ncol = 2))


In [ ]:


df_temp  <- df_totals %>% filter(totalAbundance >= min_abundance) %>% select(Group,totalAbundance)

df_filtered <- df_temp %>% inner_join(df_plot, by = "Group")

options(repr.plot.width = 20, repr.plot.height = 20)
ps <- list()

for(lc in c("cropland", "grassland", "woodland_coniferous", "woodland_deciduous"))
{

  df_crosstab <- df_filtered %>%
  filter(LandCover == lc) %>% 
  select(SampleID, Group, Abundance) %>% 
    pivot_wider(names_from = Group, values_from = Abundance,values_fill = 0) %>%
    column_to_rownames("SampleID")


  cor_matrix <- cor(df_crosstab, use = "pairwise.complete.obs")  # Handle missing values


  # Convert correlation matrix into long format for ggplot
  cor_long <- as.data.frame(as.table(cor_matrix))

  # Plot the correlation heatmap
  p <-  ggplot(cor_long, aes(Var1, Var2, fill = Freq)) +
    geom_tile(color = "white") +  # White grid lines for clarity
      geom_text(aes(label = sprintf("%.2f", Freq)), color = "black", size = 5) +  # Add text label
    scale_fill_gradient2(low = "blue", mid = "white", high = "red", midpoint = 0) +  # Diverging colors
      scale_y_discrete(limits = rev(levels(cor_long$Var2))) +  # Reverse Y-axis order
    theme_minimal() +
    labs(title = label_map[lc],        
        x = "Group", y = "Group", fill = "Correlation") +
    theme(axis.text.x = element_text(angle = 45, hjust = 1),
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5),
    axis.text = element_text(size = 16))

  ps[[length(ps)+1]] = p

  print(paste(lc,":", mean(abs(cor_matrix[upper.tri(cor_matrix)]))))

  

}


do.call(grid.arrange, c(ps, ncol = 2))

In [ ]:

df_temp  <- df_totals %>% filter(totalAbundance >= min_abundance) %>% select(Group,totalAbundance)

df_filtered <- df_temp %>% inner_join(df_plot, by = "Group")

options(repr.plot.width = 20, repr.plot.height = 20)
ps <- list()



for(lc in c("cropland", "grassland", "woodland_coniferous", "woodland_deciduous"))
{
    print(lc)
    for(g in selected_groups)
    {

      df_temp <- df_filtered %>%
          filter(LandCover == lc & Group == g)

      cor_res <-  cor(df_temp %>% pull(Multiplicity), df_temp %>% pull(Abundance), method = "pearson")
      print(paste0("   ",g, ": ", cor_res, sep=""))


    }


  

}




In [ ]:
options(repr.plot.width = 15, repr.plot.height = 15)


df_plot_1 <- df_pairs %>% filter(Group %in% c("rifamycin"))  %>% mutate(MultiplicityRifamycin = Multiplicity) %>% select(LandCover, SampleID, MultiplicityRifamycin)
df_plot_2 <- df_pairs %>% filter(Group %in% c("multidrug"))%>% mutate(MultiplicityMultidrug = Multiplicity) %>% select(LandCover, SampleID, MultiplicityMultidrug)

df_plot <- df_plot_1 %>% inner_join(df_plot_2, by = c("LandCover","SampleID"))

#df_plot <- df_plot %>% filter(LandCover %in% c("cropland","woodland_coniferous"))


# Define label mapping
label_map <- c(
  "woodland_deciduous" = "Woodland Deciduous",
  "woodland_coniferous" = "Woodland Coniferous",
  "grassland" = "Grassland",
  "cropland" = "Cropland"
)

# Define custom colors
custom_colors <- c("#BE5E58", "#DAC5AA", "#5A7273", "#7D917E")
custom_colors <- setNames(custom_colors, c("cropland", "grassland", "woodland_coniferous", "woodland_deciduous"))

# Scatter plot
ggplot(df_plot, aes(x = MultiplicityRifamycin, y = MultiplicityMultidrug, color = LandCover)) +
  geom_point(size = 4, alpha = 0.9) +  # Customize point size and transparency
    geom_smooth(method = "lm", se = FALSE, linewidth = 1.2) +  # Add regression lines without confidence interval
    scale_color_manual(values = custom_colors) +  # Apply custom colors
  labs(title = "Scatter Plot of Multiplicities by Group",
       x = "Multiplicity Rifamycin",
       y = "Multiplicity Multidrug",
       color = "Land Cover") +  # Legend title
  theme_minimal(base_size = 16)  # Use a clean themeb

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 15)


group <- "rifamycin"


df_plot <- df_pairs %>% filter(Group %in% c(group))


# Define label mapping
label_map <- c(
  "woodland_deciduous" = "Woodland Deciduous",
  "woodland_coniferous" = "Woodland Coniferous",
  "grassland" = "Grassland",
  "cropland" = "Cropland"
)

# Define custom colors
custom_colors <- c("#BE5E58", "#DAC5AA", "#5A7273", "#7D917E")
custom_colors <- setNames(custom_colors, c("cropland", "grassland", "woodland_coniferous", "woodland_deciduous"))

# Scatter plot
ggplot(df_plot, aes(x = Abundance, y = Multiplicity, color = LandCover)) +
  geom_point(size = 4, alpha = 0.9) +  # Customize point size and transparency
      geom_smooth(method = "lm", se = FALSE, linewidth = 1.2) +  # Add regression lines without confidence interval
    scale_color_manual(values = custom_colors) +  # Apply custom colors
  labs(title = paste0(str_to_title(group), " Scatter Plot of Abundance vs. Multiplicity"),
       x = "Abundance",
       y = "Multiplicity",
       color = "Land Cover") +  # Legend title
  theme_minimal(base_size = 22)  # Use a clean themeb

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 15)


group <- "multidrug"


df_plot <- df_pairs %>% filter(Group %in% c(group))


# Define label mapping
label_map <- c(
  "woodland_deciduous" = "Woodland Deciduous",
  "woodland_coniferous" = "Woodland Coniferous",
  "grassland" = "Grassland",
  "cropland" = "Cropland"
)

# Define custom colors
custom_colors <- c("#BE5E58", "#DAC5AA", "#5A7273", "#7D917E")
custom_colors <- setNames(custom_colors, c("cropland", "grassland", "woodland_coniferous", "woodland_deciduous"))

# Scatter plot
ggplot(df_plot, aes(x = Abundance, y = Multiplicity, color = LandCover)) +
  geom_point(size = 4, alpha = 0.9) +  # Customize point size and transparency
      geom_smooth(method = "lm", se = FALSE, linewidth = 1.2) +  # Add regression lines without confidence interval
    scale_color_manual(values = custom_colors) +  # Apply custom colors
  labs(title = paste0(str_to_title(group), " Scatter Plot of Abundance vs. Multiplicity"),
       x = "Abundance",
       y = "Multiplicity",
       color = "Land Cover") +  # Legend title
  theme_minimal(base_size = 22)  # Use a clean themeb

In [ ]:
library(purrr)


options(repr.plot.width = 13, repr.plot.height = 20)
ps <- list()

custom_colors <- colorRampPalette(c("white", "red"))(100)


df_crosstab <- df_filtered %>%
select(SampleID, Group, Multiplicity) %>% 
pivot_wider(names_from = Group, values_from = Multiplicity,values_fill = 0) %>%
column_to_rownames("SampleID")

for (col in names(df_crosstab)) {
    f <- ecdf(df_crosstab[, col])
    df_crosstab[, col] <- f(df_crosstab[, col]) 
}

results <- c()

for(i in 1:nrow(df_crosstab))
{
  row_name <-   rownames(df_crosstab)[i]
  results <- rbind(results, c(row_name, sort(as.vector(as.numeric(df_crosstab[i,])), decreasing = TRUE)))
}



vars  <- sapply(1:ncol(df_crosstab), function(x){paste0("Var",x)})

colnames(results)  <- c("SampleID",vars)

df_sorted <- as_tibble(results)

for(v in vars)
{
  df_sorted <- df_sorted %>% mutate(!!v := as.numeric(.data[[v]]))
}

df_sorted <- df_sorted %>%
  pivot_longer(cols = -SampleID, names_to = "Var", values_to = "Value")  %>% 
  inner_join(df_meta)



# Sorts
df_plot <- df_sorted %>%
  mutate(Var = factor(Var, levels = vars))



# Define label mapping
label_map <- c(
  "woodland_deciduous" = "Woodland Deciduous",
  "woodland_coniferous" = "Woodland Coniferous",
  "grassland" = "Grassland",
  "cropland" = "Cropland"
)

# Define custom colors
custom_colors <- c("#BE5E58", "#DAC5AA", "#5A7273", "#7D917E")
custom_colors <- setNames(custom_colors, c("cropland", "grassland", "woodland_coniferous", "woodland_deciduous"))


# Calculate horizontal line positions dynamically
hline_positions <- df_plot %>%
  group_by(Var) %>%
  summarise(y_max = max(Value), .groups = "drop") %>%
  mutate(y_sep = lead(y_max))  # Get separation lines


ggplot(df_plot, aes(x = "", y = Value, fill = LandCover)) +  # Empty x-axis for boxplots
  geom_boxplot() +
     #geom_jitter(width = 0.1, alpha = 0.25, color = "#3A3A3A") +
  geom_hline(data = hline_positions, aes(yintercept = y_max), linetype = "dashed", color = "gray50", size = 0.7, na.rm = TRUE) +
  facet_grid(Var ~ LandCover,  labeller = labeller(LandCover = label_map), scales = "free_y") +  # Create a grid layout
    scale_fill_manual(values = custom_colors) +  # Apply custom colors

  labs(x = "Land Cover", y = "Value", title = paste("Value by",target_level ,"and Land Cover (", target_level,"comparisson )")) +
    theme_minimal(base_size = 16) +  # Increase text size
  theme(
    strip.text = element_text(size = 15),  # Larger facet labels
    axis.text = element_text(size = 12),   # Larger axis labels
    strip.text.y = element_text(size = 15, face = "bold", angle = 0, hjust = 0),  # Rotate labels horizontally
    plot.title = element_text(size = 18, face = "bold"))  # Bigger title



In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
ps <- list()

custom_colors <- colorRampPalette(c("white", "red"))(100)

for(lc in c("cropland", "grassland", "woodland_coniferous", "woodland_deciduous"))
{

  df_crosstab <- df_filtered %>%
  filter(LandCover == lc) %>% 
  select(SampleID, Group, Multiplicity) %>% 
    pivot_wider(names_from = Group, values_from = Multiplicity,values_fill = 0) %>%
    column_to_rownames("SampleID")

    for (col in names(df_crosstab)) {
      f <- ecdf(df_crosstab[, col])
      df_crosstab[, col] <- f(df_crosstab[, col]) 
    }

    df_crosstab  <- df_crosstab  %>% arrange(across(everything()))

  ps[[length(ps)+1]] = pheatmap(df_crosstab, scale = "none", cluster_rows = FALSE, cluster_cols = FALSE, color = custom_colors)
}



In [ ]:
write.csv(df, "multiplicity_by_gene.csv", row.names = FALSE)